In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

# 웹 드라이버 초기화
driver = webdriver.Chrome()  # 크롬 드라이버를 사용하고 있다고 가정합니다.
driver.maximize_window()

# 페이지 열기
url = "https://www.empressgin.com/signature-cocktails/"
driver.get(url)

time.sleep(10)
# 팝업창 처리
try:
    popup = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="age_verify"]')))
    popup.click()
except:
    pass

# 스크롤을 하여 모든 컨텐츠 로드
SCROLL_PAUSE_TIME = 2

# 필요한 링크 추출
cocktail_links = driver.find_elements(By.CSS_SELECTOR, 'div.grid_container [data-number]')

# 딕셔너리 초기화
cocktail_data = []

for index, link in enumerate(cocktail_links):
    cocktail_url = link.get_attribute("href")
    
    # 링크를 열어 페이지에 들어가기
    driver.get(cocktail_url)
    
    # 페이지 로딩을 위해 시간 대기
    time.sleep(10)
    
    # 페이지 소스 가져오기
    page_source = driver.page_source
    
    # BeautifulSoup을 사용하여 데이터 추출
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Cocktail Name 추출
    h3_element = soup.select_one('#recipes_slider > div > ul > li > div > header > h3')
    if h3_element:
        cocktail_name = h3_element.get_text(strip=True)
    else:
        cocktail_name = "N/A"
    print("Cocktail Name:", cocktail_name)
    
    # Ingredients 추출
    ingredients_element = soup.find('div', class_='ingredients')
    if ingredients_element:
        ingredients = []
        item_elements = ingredients_element.find_all('div', class_='item')
        for item in item_elements:
            ingredient = item.get_text(strip=True)
            ingredients.append(ingredient)
        print("Ingredients:", ingredients)
    
    # Method 추출
    method_element = soup.select_one('#recipes_slider > div > ul > li > div > div > div:nth-of-type(3)')
    if method_element:
        method = method_element.get_text(strip=True, separator='\n')
        print("Method:", method)
    
    # 데이터 저장
    cocktail_data.append({
        "name": cocktail_name,
        "ingredients": ingredients,
        "method": method
    })
    
    # 페이지에서 뒤로 가기
    driver.execute_script("window.history.go(-1);")
    
    time.sleep(10)
# 데이터 출력
print("Cocktail Data:", cocktail_data)

# 웹 드라이버 종료
driver.quit()


Cocktail Name: EMPRESS & TONIC
Ingredients: ['2OZEmpress 1908 gin', '3OZPremium Tonic Water', 'Grapefruit Slice']
Method: Method
Fill a copa glass with ice, add Empress 1908 Gin, and serve with tonic on the side. Garnish with a grapefruit slice.
Cocktail Name: EMPRESS MARTINI
Ingredients: ['2OZEmpress 1908 Gin', '½OZLillet Blanc', 'Grapefruit Twist']
Method: Method
Stir on ice, strain into a chilled martini glass. Garnish with a grapefruit twist.
Cocktail Name: EMPRESS 75
Ingredients: ['1½OZEmpress 1908 Gin', '¾OZFresh Lemon Juice', '½OZSimple Syrup', 'Sparkling Wine', 'Lemon Twist']
Method: Method
Shake on ice, fine strain into a chilled flute, top with sparkling wine. Garnish with a lemon twist.
Cocktail Name: WINDSOR GARDEN
Ingredients: ['2OZEmpress 1908 Gin', '½OZFresh Lemon Juice', '½OZSimple Syrup', 'Basil, Thyme, Rosemary & Mint Leaves']
Method: Method
Lightly muddle herbs and shake ingredients over ice. Strain into a stemless wine glass with ice. Garnish with a sprig of rosemar

In [21]:
import json

In [22]:
with open("./cocktail_data4.json", "w", encoding="utf-8") as json_file:
    json.dump(cocktail_data, json_file, ensure_ascii=False, indent=4)